In [1]:
# For reading from partitioned cloud storage paths.
import pyarrow.dataset as pds
import pyarrow.parquet as pq
import pandas as pd
import time, dateutil
from dateutil.relativedelta import relativedelta

# Define some global variables.
s3_partitions = pds.dataset(
    "s3://anonymous@air-example-data/ursa-labs-taxi-data/by_year/2018/",
    partitioning=["month"],
)
models_to_train = [f"s3://anonymous@{file}" for file in s3_partitions.files]
SAMPLE_UNIQUE_ID = 141
FORECAST_LENGTH = 28

In [2]:
import ray

if ray.is_initialized():
    ray.shutdown()
ray.init()

2023-01-12 16:43:37,581	INFO worker.py:1230 -- Using address localhost:9031 set in the environment variable RAY_ADDRESS
2023-01-12 16:43:37,582	INFO worker.py:1352 -- Connecting to existing Ray cluster at address: 172.31.174.66:9031...
2023-01-12 16:43:37,588	INFO worker.py:1529 -- Connected to Ray cluster. View the dashboard at https://console.anyscale-staging.com/api/v2/sessions/ses_b5q8xHd42BTdukSgFqTxejLT/services?redirect_to=dashboard 


Python version:,3.8.13
Ray version:,2.2.0
Dashboard:,http://console.anyscale-staging.com/api/v2/sessions/ses_b5q8xHd42BTdukSgFqTxejLT/services?redirect_to=dashboard


In [3]:
print(ray.cluster_resources())

{'CPU': 24.0, 'memory': 66380792627.0, 'object_store_memory': 27585344716.0, 'node:172.31.174.66': 1.0, 'node:172.31.206.127': 1.0}


In [4]:
# # Test read data
# file_path = models_to_train[0]
# print(f"reading file: {file_path}")
# data = pq.read_table(
#     file_path,
#     filters=[ ("pickup_location_id", "=", SAMPLE_UNIQUE_ID) ],
#     columns=[ "pickup_at", "dropoff_at", "pickup_location_id", "dropoff_location_id", 
#               "passenger_count", "trip_distance", "fare_amount" ],
# ).to_pandas()
# data["ds"] = data["pickup_at"].dt.to_period("D").dt.to_timestamp()
# data.rename(columns={"trip_distance": "y"}, inplace=True)
# print(data.head(2))

In [5]:
# data.shape

In [6]:
# # Test train your model
# print(f"training model...")
# import prophet
# from prophet import Prophet
# # Split data into train, test.
# train_end = data.ds.max() - relativedelta(days=FORECAST_LENGTH - 1)
# train_df = data.loc[(data.ds <= train_end), :].copy()
# test_df = data.iloc[-(FORECAST_LENGTH):, :].copy()
# print(train_df.shape, test_df.shape)

# # Define Prophet model with 75% confidence interval.
# model = Prophet(interval_width=0.75, seasonality_mode="multiplicative") 
# print(type(model))

# # Train and fit Prophet model.
# model = model.fit(train_df[["ds", "y"]])
# print(type(model))

In [7]:
def train_model(file_path: str):
    data = pq.read_table(file_path,
                         filters=[ ("pickup_location_id", "=", SAMPLE_UNIQUE_ID) ],
                         columns=[ "pickup_at", "pickup_location_id", "trip_distance" ],
                        ).to_pandas()
    # Read a pyarrow parquet S3 file.
    data["ds"] = data["pickup_at"].dt.to_period("D").dt.to_timestamp()
    data.rename(columns={"trip_distance": "y"}, inplace=True)
    data.drop("pickup_at", inplace=True, axis=1)
    unique_id = data["pickup_location_id"][0]

    # Split data into train, test.
    train_end = data.ds.max() - relativedelta(days=FORECAST_LENGTH - 1)
    train_df = data.loc[(data.ds <= train_end), :].copy()
    test_df = data.iloc[-(FORECAST_LENGTH):, :].copy()
    
    # Define Prophet model with 75% confidence interval.
    import prophet
    from prophet import Prophet
    model = Prophet(interval_width=0.75, seasonality_mode="multiplicative")      

    # Train and fit Prophet model.
    model = model.fit(train_df[["ds", "y"]])

    return train_df, test_df, model, unique_id

# Serial Python

In [8]:
# This will take much too long serially.
start = time.time()
for file in models_to_train:
    print("Training model serially", file)
    train_df, test_df, model, unique_id = train_model(file)
    print(train_df.shape, test_df.shape, type(model))

time_regular_python = time.time() - start
print(f"Total number of models: {len(models_to_train)}")
print(f"TOTAL TIME TAKEN: {time_regular_python/60:.2f} minutes")

# Total number of models: 12
# TOTAL TIME TAKEN: 2.34 minutes
# Ran on 1-node AWS cluster of m5.4xlarges

Training model serially s3://anonymous@air-example-data/ursa-labs-taxi-data/by_year/2018/01/data.parquet/4d6bc4368704460d90c92c22e05a2220_000000.parquet


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/home/ray/anaconda3/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -48.6877
(27910, 3) (28, 3) <class 'prophet.forecaster.Prophet'>
Training model serially s3://anonymous@air-example-data/ursa-labs-taxi-data/by_year/2018/02/data.parquet/e817946252d1409b93964685130e76cb_000000.parquet
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      48       66815.7   1.38389e-05        100.35      0.4352      0.4352       72   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/home/ray/anaconda3/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -233.643
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        540798   0.000386738       2374.21        0.43           1      132   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199        540804   0.000411169       665.489           1           1      258   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     213        540804   3.59852e-05       216.267           1           1      274   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
(199625, 3) (28, 3) <class 'prophet.forecaster.Prophet'>
Training model serially s3://anonymous@air-example-data/ursa-labs-taxi-data/by_year/2018/03/data.parquet/0b7e5121a4904c64be5e91ceec0eee2f_000000.parquet


...

Initial log joint probability = -119.679
(65543, 3) (28, 3) <class 'prophet.forecaster.Prophet'>
Training model serially s3://anonymous@air-example-data/ursa-labs-taxi-data/by_year/2018/11/data.parquet/9c3fe546f3d746eeb3225b8150fb26e6_000000.parquet
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        154347    0.00161459       352.178           1           1      136   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199        154348   8.56524e-05       111.334      0.1246      0.1246      278   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     203        154348   1.11388e-05       225.793   9.141e-08       0.001      331  LS failed, Hessian reset 
     254        154348    8.4487e-07       127.579     0.07411     0.07411      405   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolera

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -17.4955
(28435, 3) (28, 3) <class 'prophet.forecaster.Prophet'>
Training model serially s3://anonymous@air-example-data/ursa-labs-taxi-data/by_year/2018/12/data.parquet/d9829239c5d34340a7d9ba256917ed98_000000.parquet
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      62       85944.2   2.82334e-05       54.3035      0.6341      0.6341       81   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/home/ray/anaconda3/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -25.9776
(34718, 3) (28, 3) <class 'prophet.forecaster.Prophet'>
Total number of models: 12
TOTAL TIME TAKEN: 2.36 minutes
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      33       98233.1   1.05682e-05       158.468      0.4022           1       67   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


# Ray Multiprocessing

In [9]:
len(models_to_train)

12

In [ ]:
from ray.util.multiprocessing import Pool
import tqdm

start = time.time()
# Create a pool, where each worker is assigned 1 CPU by Ray.
pool = Pool(ray_remote_args={"num_cpus": 1})

# Use the pool to run `train_model` on the data, in batches of 1.
iterator = pool.imap_unordered(train_model, models_to_train, chunksize=1)

# Track the progress using tqdm and retrieve the results into a list.
results = list(tqdm.tqdm(iterator, total=len(models_to_train)))

time_ray_multiprocessing = time.time() - start
print(f"Total number of models: {len(results)}")
print(f"TOTAL TIME TAKEN: {time_ray_multiprocessing/60:.2f} minutes")

assert len(results) == len(models_to_train)
print(type(results[0][0]), type(results[0][1]), type(results[0][2]), type(results[0][3]))

# # Chunksize=1
# Total number of models: 12
# TOTAL TIME TAKEN: 0.60 minutes


(scheduler +10m44s) Adding 1 node(s) of type worker-node-type-0.
(scheduler +13m49s) Resized to 72 CPUs.
(scheduler +14m2s) Adding 1 node(s) of type worker-node-type-0.


  0%|          | 0/12 [00:00<?, ?it/s](PoolActor pid=7270, ip=172.31.176.112) INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
(PoolActor pid=7270, ip=172.31.176.112) INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
(PoolActor pid=7270, ip=172.31.176.112) INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
(PoolActor pid=7268, ip=172.31.176.112) INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
(PoolActor pid=7271, ip=172.31.176.112) INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
(PoolActor pid=5130, ip=172.31.170.171) INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


(PoolActor pid=7270, ip=172.31.176.112) Initial log joint probability = -17.4955
(PoolActor pid=7270, ip=172.31.176.112)     Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 


(PoolActor pid=7269, ip=172.31.176.112) INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
(PoolActor pid=5129, ip=172.31.170.171) INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
(PoolActor pid=5125, ip=172.31.170.171) INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
(PoolActor pid=5125, ip=172.31.170.171) INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
(PoolActor pid=5124, ip=172.31.170.171) INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
(PoolActor pid=5126, ip=172.31.170.171) INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


(PoolActor pid=7271, ip=172.31.176.112) Initial log joint probability = -25.9776
(PoolActor pid=7271, ip=172.31.176.112)     Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
(PoolActor pid=5124, ip=172.31.170.171) Initial log joint probability = -48.6877
(PoolActor pid=5124, ip=172.31.170.171)     Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
(PoolActor pid=5126, ip=172.31.170.171) Initial log joint probability = -152.634
(PoolActor pid=5126, ip=172.31.170.171)     Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 


...

(PoolActor pid=5131, ip=172.31.170.171)       53        499170   0.000103375       151.314           1           1       91   
(PoolActor pid=5131, ip=172.31.170.171) Optimization terminated normally: 
(PoolActor pid=5131, ip=172.31.170.171)   Convergence detected: relative gradient magnitude is below tolerance


 75%|███████▌  | 9/12 [00:17<00:07,  2.36s/it]

(PoolActor pid=7268, ip=172.31.176.112)       64        497590   0.000354387       197.759           1           1       97   
(PoolActor pid=7268, ip=172.31.176.112) Optimization terminated normally: 
(PoolActor pid=7268, ip=172.31.176.112)   Convergence detected: relative gradient magnitude is below tolerance
(PoolActor pid=7269, ip=172.31.176.112)      203        154348   1.11388e-05       225.793   9.141e-08       0.001      331  LS failed, Hessian reset 
(PoolActor pid=5125, ip=172.31.170.171)       99        540798   0.000386738       2374.21        0.43           1      132   
(PoolActor pid=5125, ip=172.31.170.171)     Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
(scheduler +17m49s) Resized to 88 CPUs.


 83%|████████▎ | 10/12 [00:21<00:05,  2.93s/it]

(PoolActor pid=7269, ip=172.31.176.112)      254        154348    8.4487e-07       127.579     0.07411     0.07411      405   
(PoolActor pid=7269, ip=172.31.176.112) Optimization terminated normally: 
(PoolActor pid=7269, ip=172.31.176.112)   Convergence detected: relative gradient magnitude is below tolerance


In [ ]:
# Calculate the speed-up between serial Python and Ray Multiprocessing Pool
import numpy as np

speedup = time_regular_python / time_ray_multiprocessing
print(f"Speedup from running Ray Multiprocessing vs serial Python: {np.round(speedup, 1)}x"
      f", or {(np.round(speedup, 0)-1) * 100}%")

# Speedup from running Ray Multiprocessing vs serial Python: 3.7x, or 300.0%

# Use Ray Tune with Ray AIR

In [12]:
import numpy as np
from typing import Tuple
from ray import air, tune
from ray.air import session, ScalingConfig
from ray.air.checkpoint import Checkpoint
RAY_IGNORE_UNHANDLED_ERRORS=1

def evaluate_model_prophet(
    model: "prophet.forecaster.Prophet",
) -> Tuple[float, pd.DataFrame]:

    # Inference model using FORECAST_LENGTH.
    future_dates = model.make_future_dataframe(
        periods=FORECAST_LENGTH, freq="D"
    )
    future = model.predict(future_dates)

    # Calculate mean absolute error.
    temp = future.copy()
    temp["forecast_error"] = np.abs(temp["yhat"] - temp["trend"])
    error = np.mean(temp["forecast_error"])

    return error, future


# This function is exactly the same as train_model(), except:
# - Change the input parameter to config type dict.
# - Change the model based on 'algorithm' input parameter
def trainable_func(config: dict):
    # Read a pyarrow parquet S3 file.
    data = pq.read_table(config["file_path"],
                         filters=[ ("pickup_location_id", "=", SAMPLE_UNIQUE_ID) ],
                         columns=[ "pickup_at", "pickup_location_id", "trip_distance" ],
                        ).to_pandas()
    data["ds"] = data["pickup_at"].dt.to_period("D").dt.to_timestamp()
    data.rename(columns={"trip_distance": "y"}, inplace=True)
    data.drop("pickup_at", inplace=True, axis=1)
    unique_id = data["pickup_location_id"][0]

    # Split data into train, test.
    train_end = data.ds.max() - relativedelta(days=FORECAST_LENGTH - 1)
    train_df = data.loc[(data.ds <= train_end), :].copy()
    test_df = data.iloc[-(FORECAST_LENGTH):, :].copy()
    
    # Define Prophet model with 75% confidence interval.
    import prophet
    from prophet import Prophet
    if config["algorithm"] == "prophet_additive":
        model = Prophet(interval_width=0.75, seasonality_mode="additive")
    elif config["algorithm"] == "prophet_multiplicative":
        model = Prophet(interval_width=0.75, seasonality_mode="multiplicative")

    # Train and fit the Prophet model.
    model = model.fit(train_df[["ds", "y"]])

    # Inference model and evaluate error.
    error, future = evaluate_model_prophet(model)

    # Define a model checkpoint using AIR API.
    # https://docs.ray.io/en/latest/tune/tutorials/tune-checkpoints.html
    checkpoint = ray.air.checkpoint.Checkpoint.from_dict(
        {
            "model": model,
            "forecast_df": future,
            "location_id": unique_id,
        }
    )

    # Save checkpoint and report back metrics, using ray.air.session.report()
    metrics = dict(error=error)
    session.report(metrics, checkpoint=checkpoint)

In [13]:
# Tune is designed for up to thousands of trials.
param_space = {
    "file_path": tune.grid_search(
        [ f"s3://anonymous@{file}" for file in s3_partitions.files ]
    ),
    "algorithm": tune.grid_search(["prophet_additive", "prophet_multiplicative"]),
}

In [16]:
start = time.time()
tuner = tune.Tuner(
        trainable_func,
        param_space=param_space,
        # tune_config=tune.TuneConfig(
        #     metric="error",
        #     mode="min",
        # ),
        # run_config=air.RunConfig(
        #     # Redirect logs to relative path instead of default ~/ray_results/.
        #     local_dir="my_Tune_logs",
        #     # Specify name to make logs easier to find in log path.
        #     name="tune_nyc",
        # ),
    )
results = tuner.fit()

time_ray_tune = time.time() - start
print(f"Total number of models: {len(results)}")
print(f"TOTAL TIME TAKEN: {time_ray_tune/60:.2f} minutes")

# Total number of models: 24
# TOTAL TIME TAKEN: 1.17 minutes

(trainable_func pid=2347, ip=172.31.170.171) INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


(trainable_func pid=2347, ip=172.31.170.171) Initial log joint probability = -48.6877
(trainable_func pid=2347, ip=172.31.170.171)     Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
(trainable_func pid=2347, ip=172.31.170.171)       20       66815.8   1.94239e-06        105.79      0.3437           1       33   
(trainable_func pid=2347, ip=172.31.170.171) Optimization terminated normally: 
(trainable_func pid=2347, ip=172.31.170.171)   Convergence detected: relative gradient magnitude is below tolerance


(trainable_func pid=4850, ip=172.31.176.112) INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
(trainable_func pid=4850, ip=172.31.176.112) INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
(trainable_func pid=4850, ip=172.31.176.112) INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


(trainable_func pid=4850, ip=172.31.176.112) Initial log joint probability = -9.08708
(trainable_func pid=4850, ip=172.31.176.112)     Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 


(trainable_func pid=4846, ip=172.31.176.112) INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
(trainable_func pid=2750, ip=172.31.170.171) INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
(trainable_func pid=2749, ip=172.31.170.171) INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
(trainable_func pid=4849, ip=172.31.176.112) INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
(trainable_func pid=4847, ip=172.31.176.112) INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
(trainable_func pid=4847, ip=172.31.176.112) INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
(trainable_func pid=2754, ip=172.31.170.171) INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True t

...

(trainable_func pid=4848, ip=172.31.176.112)      213        540804   3.59852e-05       216.267           1           1      274   
(trainable_func pid=4848, ip=172.31.176.112) Optimization terminated normally: 
(trainable_func pid=4848, ip=172.31.176.112)   Convergence detected: relative gradient magnitude is below tolerance


2023-01-12 16:52:40,115	WARNING util.py:244 -- The `process_trial_save` operation took 1.415 s, which may be a performance bottleneck.
2023-01-12 16:52:41,385	WARNING util.py:244 -- The `process_trial_save` operation took 1.240 s, which may be a performance bottleneck.
2023-01-12 16:52:42,544	WARNING util.py:244 -- The `process_trial_save` operation took 1.157 s, which may be a performance bottleneck.


(trainable_func pid=4851, ip=172.31.176.112)      178        492635   2.39523e-05       347.982           1           1      211   
(trainable_func pid=4851, ip=172.31.176.112) Optimization terminated normally: 
(trainable_func pid=4851, ip=172.31.176.112)   Convergence detected: relative gradient magnitude is below tolerance


2023-01-12 16:52:45,097	WARNING util.py:244 -- The `process_trial_save` operation took 1.394 s, which may be a performance bottleneck.
2023-01-12 16:52:46,226	WARNING util.py:244 -- The `process_trial_save` operation took 1.127 s, which may be a performance bottleneck.
2023-01-12 16:52:48,900	WARNING util.py:244 -- The `process_trial_save` operation took 1.511 s, which may be a performance bottleneck.
2023-01-12 16:52:52,691	WARNING util.py:244 -- The `process_trial_save` operation took 2.590 s, which may be a performance bottleneck.
2023-01-12 16:52:54,960	WARNING util.py:244 -- The `process_trial_save` operation took 1.075 s, which may be a performance bottleneck.
2023-01-12 16:52:56,134	WARNING util.py:244 -- The `process_trial_save` operation took 1.162 s, which may be a performance bottleneck.
2023-01-12 16:53:02,150	WARNING util.py:244 -- The `process_trial_save` operation took 1.253 s, which may be a performance bottleneck.
2023-01-12 16:53:03,334	WARNING util.py:244 -- The `pro

Total number of models: 24
TOTAL TIME TAKEN: 1.42 minutes


In [18]:
# Calculate the speed-up between serial Python and Ray Tune

# Trained 2x number of models this time compared to original serial Python.
speedup = time_regular_python / time_ray_tune * 2
print(f"Speedup from running Ray Tune with AIR vs serial Python: {np.round(speedup, 1)}x"
      f", or {(np.round(speedup, 0)-1) * 100}%")

# Speedup from running Ray Tune with AIR vs serial Python: 242.3x, or 24100.0%

Speedup from running Ray Tune with AIR vs serial Python: 242.3x, or 24100.0%
